# Intial Setup

In [35]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [36]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2022-04-11 16:23:59--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.39.144
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.39.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-04-11 16:24:00--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?OYqjE9VGUp27dY-srBn3LBN_0GHxSBRlDxfntyZ6W-OzhQE49zrnZrpKumFEe-37XMPnW6LdfJ5Fq1JNg0du-G43cUgq0B96ynXkF-CNvgD_s

In [37]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [38]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ylm6j18o
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ylm6j18o


In [40]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


# Bubble Sort

In [60]:
%%cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <device_functions.h>
#include <iostream>
#include <stdio.h>
#include <stdlib.h>
#include <chrono>
#include <atomic>
#include <thread>
#include <cuda_profiler_api.h>

using namespace std;

#define SIZE 1024 // FOR PARALEL GPU  SIZE HAS TO BE 2^n 1024
#define THREADS 4 // FOR PARALEL GPU  THREADS = SIZE / (BLOCKS * 2) 
#define BLOCKS 128//FOR PARALEL GPU  BLOCKS = SIZE / (THREADS * 2) 
string type = "DEVICE";  // USE "HOST" FOR CPU BUBBLE SORT, USE "DEVICE" FOR GPU BUBBLE SORT
int flag = 0;


__host__ void bubbleSortHost(int *array, int index)
{
	int temp;
	do {

		for (int i = 0; i < SIZE - 1 - index * 2 - flag; i++) {
			if (array[index * 2 + i] > array[index * 2 + 1 + i]) {
				temp = array[index * 2 + 1 + i];
				array[index * 2 + 1 + i] = array[index * 2 + i];
				array[index * 2 + i] = temp;
			}
		}

		flag++;

	} while (SIZE - 1 - index * 2 - flag> 0);
}


__global__ void bubbleSortDeviceParallel(int *array, int offSet)
{

	int index = blockIdx.x * blockDim.x + threadIdx.x;
	int indexPerBlock = threadIdx.x;
	int temp;

	if (index  < THREADS* BLOCKS) {

		// FIRST STEP
		if (offSet == 0) {

			// DO THREAD SORTING IN CORRESPONDING BLOCK 
			for (int j = 0; j < THREADS / 2; j++) {

				for (int i = 0; i < THREADS * 2 - 1 - indexPerBlock * 2; i++) {

					if (array[index * 2 + i] > array[index * 2 + 1 + i]) {
						temp = array[index * 2 + 1 + i];
						array[index * 2 + 1 + i] = array[index * 2 + i];
						array[index * 2 + i] = temp;
					}
				}
				__syncthreads();
			}
		}
		// ALL OTHER STEPS, INDEX/THREADS/BLOCKS SHIFTED FOR int offSet
		// LAST BLOCK SKIPPED
		else {
			if (blockIdx.x != BLOCKS - 1) {
				for (int j = 0; j < THREADS / 2; j++) {
					for (int i = offSet; i < THREADS * 2 - 1 + offSet - indexPerBlock * 2; i++) {

						if (array[index * 2 + i] > array[index * 2 + 1 + i]) {
							temp = array[index * 2 + 1 + i];
							array[index * 2 + 1 + i] = array[index * 2 + i];
							array[index * 2 + i] = temp;
						}

					}
					__syncthreads();
				}
			}
		}
	}

}

int main()
{
	srand(time(NULL));
	int h_count = SIZE;
	int counter = BLOCKS;
	int *h_array;
	int *d_array;
	int offSet;

	h_array = new int[h_count];

	


	// GENERATING RANDOM ELEMENTS FOR h_array
	
	for (int i = 0; i < h_count; i++) {
	h_array[i] = rand() % SIZE;
	}


	// BUBBLE SORT USING GPU

		cout << "ELEMENTS OF ARRAY BEFORE SORT: " << endl;
		for (int i = 0; i < SIZE; i++)
		{
			cout << h_array[i] << " ";
		}
		cout << endl;

		if (cudaMalloc(&d_array, sizeof(int) * h_count) != cudaSuccess)
		{
			cout << "D_ARRAY ALLOCATING NOT WORKING!" << endl;
			return 0;
		}

		if (cudaMemcpy(d_array, h_array, sizeof(int)* h_count, cudaMemcpyHostToDevice) != cudaSuccess)
		{
			cout << "cudaMemcpyHostToDevice ERROR!" << endl;
			cudaFree(d_array);
			return 0;
		}

		cudaEvent_t beginEvent;
		cudaEvent_t endEvent;

		cudaEventCreate(&beginEvent);
		cudaEventCreate(&endEvent);

		cudaEventRecord(beginEvent);

		do {

			for (int i = 0; i < THREADS * 2; i++) {
				offSet = i;
				// POSSIBLE CHANGE: if offset != 0 USE bubbleSortDeviceParallel <<< BLOCKS-1, THREADS >>> (d_array, offSet);
				bubbleSortDeviceParallel <<< BLOCKS, THREADS >>> (d_array, offSet);
			}

			counter--;
		} while (counter > 0);

		cudaDeviceSynchronize();
		cudaEventRecord(endEvent);
		cudaEventSynchronize(endEvent);

		float timeValue = 0;
		cudaEventElapsedTime(&timeValue, beginEvent, endEvent);

		

		if (cudaMemcpy(h_array, d_array, sizeof(int)* h_count, cudaMemcpyDeviceToHost) != cudaSuccess)
		{
			delete[] h_array;
			cudaFree(d_array);
			cout << "cudaMemcpyDeviceToHost Error" << endl;
			system("pause");
			return 0;
		}
		cout << endl;

		// ARRAY AFTER BUBBLE SORT
		cout << "BUBBLE SORT RESULTS: " << endl;
		for (int i = 0; i < h_count; i++) {
			cout << h_array[i] << " ";
		}
		cout << endl;


	// FREEING MEMORY OF CPU & GPU
	delete[] h_array;
	cudaFree(d_array);
	cudaDeviceReset();

	
	return 0;
}

ELEMENTS OF ARRAY BEFORE SORT: 
104 881 228 359 886 898 846 1001 319 184 316 19 938 580 877 213 681 247 863 390 15 71 781 145 354 50 207 1023 124 348 863 228 206 67 587 68 965 409 45 260 594 362 279 508 942 133 721 600 380 561 990 395 632 747 541 986 797 748 985 921 72 824 126 278 891 713 347 832 99 392 69 693 754 348 177 673 481 898 249 861 435 215 233 43 962 774 6 735 498 991 632 570 792 758 849 659 448 172 468 547 564 537 216 295 885 393 968 343 267 193 180 703 408 413 746 346 163 752 57 661 720 689 208 488 424 33 123 872 205 591 395 769 104 611 40 990 1004 1008 309 247 177 489 950 585 903 673 931 42 401 988 704 97 654 912 585 54 945 709 926 126 276 297 895 381 908 936 347 888 920 656 111 74 121 38 659 0 711 567 43 88 531 747 186 161 635 771 215 556 456 117 682 733 414 553 90 298 465 437 162 362 69 274 436 190 312 71 191 1023 638 234 87 146 981 273 307 592 21 523 124 477 640 806 186 31 335 276 329 801 713 492 139 782 766 575 973 54 646 140 53 261 374 140 407 331 414 714 923 435 213 

# Count Sort

In [61]:
%%cu

#include <cstdio>
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <math.h>
#define MAX_VALUE 256
#define BLOCKS 32
#define THREADS 128


__global__ void count(int *A, int *B, int n) {

	int b_id 		= blockIdx.x,
			b_num 	= gridDim.x,
			b_size,
			b_offset,
			t_id 	= threadIdx.x,
			t_num 	= blockDim.x,
			t_size,
			t_offset,
			offset;

	// initialize a shared memory array to store the count for each block.
	__shared__ int count[MAX_VALUE];

	// set intial values to zeros. Each thread sets its own share to zero.
	t_size = (t_num > MAX_VALUE ? 1 : MAX_VALUE / t_num);
	offset = t_id * t_size;
	for (int i = offset; i < offset + t_size && i < MAX_VALUE; ++i)
		count[i] = 0;

	// wait until all threads have completed the initialization process.
	__syncthreads();

	// accumulate the counts of each value. Each thread counts a certain portain
	// of the unsorted array.
	b_size = (b_num > n ? 1 : n / b_num);
	b_offset = b_id * b_size;

	t_size = (t_num > b_size ? 1 : b_size / t_num);

	offset = b_offset + t_id * t_size;
	for (int i = offset; i < offset + t_size && i < b_offset + b_size && i < n; ++i)
		atomicAdd(&count[A[i]], 1);

	// wait until all threads have completed the couting phase.
	__syncthreads();

	// copy the block count into global memory. Each thread copies its portioin to 
	// the global memory.
	t_size = (t_num > MAX_VALUE ? 1 : MAX_VALUE / t_num);
	t_offset = t_id * t_size;
	offset = b_id * MAX_VALUE + t_offset;

	if (offset + t_size <= (b_id + 1) * MAX_VALUE)
		memcpy(&B[offset], &count[t_offset], sizeof(int) * t_size);

}

__global__ void merge(int *B) {

	int b_id	= blockIdx.x,
		b_num	= gridDim.x,
		b_size,
		b_offset,
		t_id	= threadIdx.x,
		t_num	= blockDim.x,
		t_size,
		offset;

	// loop through and merge until all arrays are merged.
	for (int i = b_num, j = 2; i != 1; i /= 2, j *= 2) {

		// each block will operate on b_size values which equal, the number of 
		// count arrays * size of count arrays / number of blocks / 2. The final 2
		// represents the merge process.
		b_size = i * MAX_VALUE / b_num / 2;
		b_offset = (b_id / j) * (j * MAX_VALUE) + b_size * (b_id % j);

		t_size = (t_num > b_size ? 1 : b_size / t_num);

		// calculate the offset that each thread will start at and sum counts.
		offset = b_offset + t_id * t_size;
		for (int k = offset, l = offset + (MAX_VALUE * (j / 2)); 
			k < offset + t_size && k < b_offset + b_size; ++k, ++l)
			B[k] += B[l];

		// wait untill all arrays are merged for every step.
		__syncthreads();

	}

}

// print results.
void print(const char *name, int *array, int size) {

	printf("%s = [%d", name, array[0]);
	for (int i = 1; i < size; ++i) printf(", %d", array[i]);
	printf("]\n");

}


int main() {

	

	int n = pow(2,5),
		A[n], B[MAX_VALUE], C[n],
		*dA, *dB;

	// malloc device memory
	cudaMalloc((void**)&dA, sizeof(int) * n);
	cudaMalloc((void**)&dB, sizeof(int) * BLOCKS * MAX_VALUE);

	// initialize srand for randomizing values based on time.
	time_t time;
	srand((time)&time);

	// dump random values in array A.
	for (int i = 0; i < n; ++i)
		A[i] = rand() % MAX_VALUE;

	// copy the content of array A to dA on the device.
	cudaMemcpy(dA, A, sizeof(int) * n, cudaMemcpyHostToDevice);

	// call the count kernal.
	count<<<BLOCKS, THREADS>>>(dA, dB, n);

	// merge count ararys using the merge kernal.
	merge<<<BLOCKS, THREADS>>>(dB);

	// copy result of the summation of counted arrays to host.
	cudaMemcpy(B, dB, sizeof(int) * MAX_VALUE, cudaMemcpyDeviceToHost);

	// construct the sorted array.
	for (int i = 0, j = 0; i < MAX_VALUE; ++i)
		for (int k = 0; k < B[i]; ++k, ++j)
			C[j] = i;

	// print unsorted and sorted arrays.
	print("Original", A, n);
	print("Sorted", C, n);

	// free device memory.
	cudaFree(dA);
	cudaFree(dB);

	return EXIT_SUCCESS;

}

Original = [103, 198, 105, 115, 81, 255, 74, 236, 41, 205, 186, 171, 242, 251, 227, 70, 124, 194, 84, 248, 27, 232, 231, 141, 118, 90, 46, 99, 51, 159, 201, 154]
Sorted = [27, 41, 46, 51, 70, 74, 81, 84, 90, 99, 103, 105, 115, 118, 124, 141, 154, 159, 171, 186, 194, 198, 201, 205, 227, 231, 232, 236, 242, 248, 251, 255]

